In [1]:
from final_utils import *
from shufflenetV2_tf2 import *

In [2]:
root_path = generate_root_path()

### DataLoader and other nesseccary functions

In [3]:
def lr_exp_decay(epoch, lr):
    k = 0.04
    return lr * np.exp(-k*epoch)

def load_batch(paths, main_path=f'{root_path}Data/splitted/train/'):
    images = []
    feats = []
    for i, path in enumerate(paths):
        try:
            img = cv2.imread(path, cv2.COLOR_BGR2RGB)
            img = np.array(img)
            img = img.astype('float32')
            img /= 255 
            images.append(img)
        except:
            continue

        path_to_feat = path.split('.')[0].split('/')[-1] + '.npy'
        path_to_feat = main_path + 'features/mg/original/' + path_to_feat 
        feats.append(np.load(path_to_feat))
    
    images = np.array(images)
    feats = np.squeeze(np.array(feats))

    return images, feats

### Trainer

In [4]:
def trainer(model, data, data_val,batch_size=128, epochs=30, learning_rate=0.03):
    X_train, y_train = load_batch(data)

    history = model.fit(X_train, y_train,
                        batch_size=batch_size,
                        epochs=1,
                        verbose=0,
                        validation_data = data_val)

    return history

### Loading data

In [5]:
main_path = main_path=f'{root_path}Data//splitted/train/'
paths_bad = []
paths_good = []
    
for i in range(7):
    alm_train_bad = open(f'{main_path}data_bad{i+1}.json')
    bad_data = json.load(alm_train_bad)
    
    for data in bad_data:
        path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
        paths_bad.append(path_to_img)
        
alm_train_good = open(f'{main_path}/data_good1.json')
good_data = json.load(alm_train_good)
for data in good_data:
    path_to_img = main_path + f'images/{data["label"]}/{data["splitted"]}_resized_600x600/' + data['name']
    paths_good.append(path_to_img)
    
for i in range(7):
    paths_bad[i] = np.squeeze(np.array(paths_bad[i]))
paths_good = np.squeeze(np.array(paths_good))
   
# Generating static validation data
paths_bad, paths_bad_val = extract_static_val_data(paths_bad, perc = 0.0017) #original - 0.017
paths_good, paths_good_val = extract_static_val_data(paths_good, perc = 0.01) #original - 0.11

paths_bad = np.array(paths_bad)
paths_bad_val = np.array(paths_bad_val)
paths_good = np.array(paths_good)
paths_good_val = np.array(paths_good_val)

In [6]:
full_data = np.concatenate((np.repeat(paths_good, 7), paths_bad))
    
#shuffling
idx = np.random.permutation(len(full_data))
full_data = full_data[idx]
full_data = full_data[:10000]
full_data.shape

(10000,)

In [7]:
#Splitting data 
split_factor = 200
splitted_data = []

global_batches = int(full_data.shape[0] / split_factor)
for i in range(global_batches):
    batch_data = full_data[i*split_factor: (i+1)*split_factor]
    splitted_data.append(batch_data)
    
splitted_data[-1] = np.concatenate((splitted_data[-1], full_data[len(splitted_data)*split_factor:]))

data = splitted_data

In [8]:
len(data)

50

In [9]:
#Loading validation data
paths_val = np.concatenate( (paths_bad_val, paths_good_val ) , axis=0 )
X_val, y_val = load_batch(paths_val)

data_val = (X_val, y_val)

### Creating model and training

In [10]:
batch_size = 8
epochs = 10
learning_rate = 0.003

In [2]:
model = ShufflenetV2(num_classes=16928, training=True)

In [3]:
model.build([64, 600, 600, 3])
model.summary()

Model: "shufflenet_v2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  672       
                                                                 
 batch_normalization (BatchN  multiple                 96        
 ormalization)                                                   
                                                                 
 activation (Activation)     multiple                  0         
                                                                 
 max_pooling2d (MaxPooling2D  multiple                 0         
 )                                                               
                                                                 
 shuffle_block (ShuffleBlock  multiple                 4668      
 )                                                               
                                                     

In [12]:
model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=keras.optimizers.Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, 
                                              epsilon=1e-07, decay=0, amsgrad=False), run_eagerly=True)
for epoch in range(epochs):
    print(f'Epoch {epoch+1}/{epochs}: ')
#     if epoch != 0:
#         model.load_weights(f'models/ShuffleNet/Shufflenet_on_600x600_labels_MG_original.h5')
        
    for i in range(len(data)):
        print(10*'-', i, 10*'-')
        batch_data = np.array(data[i])
        history = trainer(model, 
                          batch_data, 
                          data_val,
                          batch_size=batch_size,
                          epochs=epochs, 
                          learning_rate=learning_rate)
#     learning_rate = lr_exp_decay(epoch+1, learning_rate)
    
    model.save_weights(f'models/ShuffleNet/Shufflenet_on_600x600_labels_MG_original.h5', save_format='h5')

Epoch 1/10: 
---------- 0 ----------
---------- 1 ----------
---------- 2 ----------
---------- 3 ----------
---------- 4 ----------


InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

### Training on dummy data

In [ ]:
img_data = tf.random.uniform((64, 600, 600, 3))
labels = np.random.randint(0,1,(64, 16928))
data = img_data, labels

In [ ]:
img_data = tf.random.uniform((4, 600, 600, 3))
labels = np.random.randint(0,1,(4, 16928))
data_val = img_data, labels

In [ ]:
type(labels)

In [ ]:
labels = tf.convert_to_tensor(labels)

In [ ]:
type(img_data)

In [ ]:
history = trainer(model, 
                  data, 
                  weights_path,
                  data_val,
                  batch_size=batch_size,
                  epochs=epochs, 
                  learning_rate=learning_rate)